In [ ]:
import pandas as pd
import os
import re 
import json
import yaml
from glob import glob
from tqdm import tqdm
from pprint import pprint
import torch
import pytorch_lightning as pl
from rouge import Rouge # 모델의 성능을 평가하기 위한 라이브러리입니다.

from torch.utils.data import Dataset , DataLoader
from transformers import AutoTokenizer, BartForConditionalGeneration, BartConfig
from transformers import Seq2SeqTrainingArguments, Seq2SeqTrainer
from transformers import Trainer, TrainingArguments
from transformers import EarlyStoppingCallback

import wandb # 모델 학습 과정을 손쉽게 Tracking하고, 시각화할 수 있는 라이브러리입니다.

# visualization
import matplotlib
matplotlib.rcParams['axes.unicode_minus'] = False
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm
fe = fm.FontEntry(
    fname=r'/usr/share/fonts/NanumFont/NanumGothic.ttf', # ttf 파일이 저장되어 있는 경로
    name='NanumBarunGothic')                        # 이 폰트의 원하는 이름 설정
fm.fontManager.ttflist.insert(0, fe)              # Matplotlib에 폰트 추가
plt.rcParams.update({'font.size': 10, 'font.family': 'NanumBarunGothic'}) # 폰트 설정
plt.rc('font', family='NanumBarunGothic')
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

# Tokenizer 정의

In [ ]:
# config 설정에 tokenizer 모듈이 사용되므로 미리 tokenizer를 정의해줍니다.
tokenizer = AutoTokenizer.from_pretrained("digit82/kobart-summarization")

# Config 정의

In [ ]:
config_data = {
    "general": {
        "data_path": "../../data", # 모델 생성에 필요한 데이터 경로를 사용자 환경에 맞게 지정합니다.
        "model_name": "digit82/kobart-summarization", # 불러올 모델의 이름을 사용자 환경에 맞게 지정할 수 있습니다.
        "output_dir": "./" # 모델의 최종 출력 값을 저장할 경로를 설정합니다.
    },
    "tokenizer": {
        "encoder_max_len": 512,
        "decoder_max_len": 100,
        "bos_token": f"{tokenizer.bos_token}",
        "eos_token": f"{tokenizer.eos_token}",
        # 특정 단어들이 분해되어 tokenization이 수행되지 않도록 special_tokens을 지정해줍니다.
        "special_tokens": ['#Person1#', '#Person2#', '#Person3#', '#PhoneNumber#', '#Address#', '#PassportNumber#']
    },
    "training": {
        "overwrite_output_dir": True,
        "num_train_epochs": 20,
        "learning_rate": 1e-5,
        "per_device_train_batch_size": 50,
        "per_device_eval_batch_size": 32,
        "warmup_ratio": 0.1,
        "weight_decay": 0.01,
        "lr_scheduler_type": 'cosine',
        "optim": 'adamw_torch',
        "gradient_accumulation_steps": 1,
        "evaluation_strategy": 'epoch',
        "save_strategy": 'epoch',
        "save_total_limit": 5,
        "fp16": True,
        "load_best_model_at_end": True,
        "seed": 42,
        "logging_dir": "./logs",
        "logging_strategy": "epoch",
        "predict_with_generate": True,
        "generation_max_length": 100,
        "do_train": True,
        "do_eval": True,
        "early_stopping_patience": 3,
        "early_stopping_threshold": 0.001,
        "report_to": "wandb" # (선택) wandb를 사용할 때 설정합니다.
    },
    # (선택) wandb 홈페이지에 가입하여 얻은 정보를 기반으로 작성합니다.
    "wandb": {
        "entity": "entity_name",
        "project": "project_name",
        "name": "name"
    },
    "inference": {
        "ckt_path": "model ckt path", # 사전 학습이 진행된 모델의 checkpoint를 저장할 경로를 설정합니다.
        "result_path": "./prediction/",
        "no_repeat_ngram_size": 2,
        "early_stopping": True,
        "generate_max_length": 100,
        "num_beams": 4,
        "batch_size" : 32,
        # 정확한 모델 평가를 위해 제거할 불필요한 생성 토큰들을 정의합니다.
        "remove_tokens": ['<usr>', f"{tokenizer.bos_token}", f"{tokenizer.eos_token}", f"{tokenizer.pad_token}"]
    }
}

In [ ]:
# 모델의 구성 정보를 YAML 파일로 저장합니다.
config_path = "./config.yaml"
with open(config_path, "w") as file:
    yaml.dump(config_data, file, allow_unicode=True)

In [ ]:
# 저장된 config 파일을 불러옵니다.
config_path = "./config.yaml"

with open(config_path, "r") as file:
    loaded_config = yaml.safe_load(file)

실험에서 쓰일 데이터를 load하여 데이터의 구조와 내용을 살펴보겠습니다.

Train, dev, test 순서대로 12457, 499, 250개 씩 데이터가 구성되어 있습니다.

In [ ]:
# config에 저장된 데이터 경로를 통해 train과 validation data를 불러옵니다.
data_path = loaded_config['general']['data_path']

# train data의 구조와 내용을 확인합니다.
train_df = pd.read_csv(os.path.join(data_path,'train.csv'))
display(train_df.tail())

# dev data의 구조와 내용을 확인합니다.
dev_df = pd.read_csv(os.path.join(data_path,'dev.csv'))
display(dev_df.tail())

# test data의 구조와 내용을 확인합니다.
test_df = pd.read_csv(os.path.join(data_path,'test.csv'))
display(test_df.tail())

# train, dev, test 길이 확인
print(train_df.shape[0], dev_df.shape[0], test_df.shape[0])

# 토픽

In [ ]:
gb = train_df.groupby('topic')['summary'].aggregate(['count']).sort_values(by='count', ascending=False).reset_index()
gb

In [ ]:
plt.figure(figsize=(12, 8))
sns.barplot(data=gb.iloc[:30], x='count', y='topic')#, palette='Set2')
plt.title(f'토픽 (상위 30개)') # 토픽 전체 6527개
plt.xlabel('Count')
plt.ylabel('Topic')
plt.show()

# 특정 토픽별 대화 흐름

## 일상 대화, 쇼핑, 전화 통화, 직업 면접, 음식 주문, 인터뷰, 길 묻기

In [ ]:
def specific_topic(df, topic_name, idx):
    """
    df : 데이터프레임 변수
    topic_name : 특정 토픽 이름
    idx : 확인하려는 개수
    """
    cond = df[df['topic'] == topic_name]
    a = cond['dialogue'].reset_index(drop=True)
    b = cond['summary'].reset_index(drop=True)
    # print(f'{topic_name}: {len(cond)}개')
    for i in range(0, idx):
        print(f'--- 대화 ---\n{a[i]}')
        print(f'--- 요약 ---\n{b[i]}\n')

In [ ]:
specific_topic(train_df, '길 묻기', 5)

# max_length 설정 등을 위한 길이 확인

## 초기 확인

In [ ]:
# kobart-summarization 기준으로 확인
dialogue_tokenizer = train_df['dialogue'].apply(lambda x: tokenizer(x))
summary_tokenizer = train_df['summary'].apply(lambda x: tokenizer(x))
topic_tokenizer = train_df['topic'].apply(lambda x: tokenizer(x))

dialogue_len = dialogue_tokenizer.apply(lambda x: len(x['input_ids']))
summary_len = summary_tokenizer.apply(lambda x: len(x['input_ids']))
topic_len = topic_tokenizer.apply(lambda x: len(x['input_ids']))

In [ ]:
# description으로 대략적인 파악
temp = dialogue_len.describe().reset_index()
temp0 = summary_len.describe().reset_index()
temp1 = topic_len.describe().reset_index()

length_dsc = temp.merge(temp0, on='index').merge(temp1, on='index')
length_dsc

In [ ]:
# scatter plot으로 분포 확인
plt.figure(figsize=(18, 4))

plt.subplot(1, 3, 1)
sns.scatterplot(data=dialogue_len.reset_index(), x=dialogue_len.index, y='dialogue')
plt.title('대화 길이')
plt.xlabel('Index')
plt.grid(True)

plt.subplot(1, 3, 2)
sns.scatterplot(data=summary_len.reset_index(), x=summary_len.index, y='summary')
plt.title('요약문 길이')
plt.xlabel('Index')
plt.grid(True)

plt.subplot(1, 3, 3)
sns.scatterplot(data=topic_len.reset_index(), x=topic_len.index, y='topic')
plt.title('토픽 길이')
plt.xlabel('Index')
plt.grid(True)

plt.tight_layout()
plt.show()

## 대화 길이보다 요약문 길이가 짧음

In [ ]:
dia_len_list = dialogue_len.tolist()
sum_len_list = summary_len.tolist()
tpc_len_list = topic_len.tolist()

for idx in range(0, train_df.shape[0]):
    if dia_len_list[idx] <= sum_len_list[idx]:
        print(idx, end=' ')

## 대화 길이는 800 밑으로 대거 분포해있음

In [ ]:
dia_len_800 = []
sum_len_125 = []
tpc_len_40 = []
for idx in range(0, train_df.shape[0]):
    if dia_len_list[idx] >= 800:
        dia_len_800.append(idx)
    if sum_len_list[idx] >= 125:
        sum_len_125.append(idx)
    if tpc_len_list[idx] >= 40:
        tpc_len_40.append(idx)

dia_800_df = train_df.loc[dia_len_800]
sum_125_df = train_df.loc[sum_len_125]
tpc_40_df = train_df.loc[tpc_len_40]

print(dia_800_df.shape[0], sum_125_df.shape[0], tpc_40_df.shape[0])

display(dia_800_df.head())
display(sum_125_df.head())
display(tpc_40_df.head())

In [ ]:
def dia_sum_check(df):
    """
    df : 데이터프레임 변수
    """
    # idx_list = df['fname'].apply(lambda x: int(x.split('_')[1])).tolist() # fname 잘 못 되어 있음
    idx_list = df.reset_index()['index'].tolist()
    for idx in idx_list:
        print(f"--- 대화 ---\n{df['dialogue'].loc[idx]}")
        print(f"--- 요약 ---\n{df['summary'].loc[idx]}\n")

dia_sum_check(dia_800_df)

In [ ]:
gb = dia_800_df.groupby('topic')['summary'].aggregate(['count']).sort_values(by='count', ascending=False).reset_index()
plt.figure(figsize=(12, 8))
sns.barplot(data=gb, x='count', y='topic')#, palette='Set2')
plt.title(f'토픽')
plt.xlabel('Count')
plt.ylabel('Topic')
plt.show()

## 요약문 길이는 125 밑으로 대거 분포해있음

In [ ]:
dia_sum_check(sum_125_df)

In [ ]:
gb = sum_125_df.groupby('topic')['summary'].aggregate(['count']).sort_values(by='count', ascending=False).reset_index()
plt.figure(figsize=(12, 8))
sns.barplot(data=gb, x='count', y='topic')#, palette='Set2')
plt.title(f'토픽')
plt.xlabel('Count')
plt.ylabel('Topic')
plt.show()

## 토픽 길이는 대부분 20 이하인데, 40 이상인 토픽 3개가 존재함

In [ ]:
dia_sum_check(tpc_40_df)

In [ ]:
for idx in tpc_40_df.reset_index()['index'].tolist():
    print(f"{idx}. {tpc_40_df['topic'][idx]}")

## 좌우 공백 제거

In [ ]:
dialogue_strip = train_df['dialogue'].apply(lambda x: x.strip())
summary_strip = train_df['summary'].apply(lambda x: x.strip())
topic_strip = train_df['topic'].apply(lambda x: x.strip())

dialogue_tokenizer_strip = dialogue_strip.apply(lambda x: tokenizer(x))
summary_tokenizer_strip = summary_strip.apply(lambda x: tokenizer(x))
topic_tokenizer_strip = topic_strip.apply(lambda x: tokenizer(x))

dialogue_strip_len = dialogue_tokenizer_strip.apply(lambda x: len(x['input_ids']))
summary_strip_len = summary_tokenizer_strip.apply(lambda x: len(x['input_ids']))
topic_strip_len = topic_tokenizer_strip.apply(lambda x: len(x['input_ids']))

In [ ]:
temp = dialogue_strip_len.describe().reset_index()
temp0 = summary_strip_len.describe().reset_index()
temp1 = topic_strip_len.describe().reset_index()

length_dsc = temp.merge(temp0, on='index').merge(temp1, on='index')
display(length_dsc)

plt.figure(figsize=(18, 4))

plt.subplot(1, 3, 1)
sns.scatterplot(data=dialogue_strip_len.reset_index(), x=dialogue_strip_len.index, y='dialogue')
plt.title('대화 길이 (공백 제거)')
plt.xlabel('Index')
plt.grid(True)

plt.subplot(1, 3, 2)
sns.scatterplot(data=summary_strip_len.reset_index(), x=summary_strip_len.index, y='summary')
plt.title('요약문 길이 (공백 제거)')
plt.xlabel('Index')
plt.grid(True)

plt.subplot(1, 3, 3)
sns.scatterplot(data=topic_strip_len.reset_index(), x=topic_strip_len.index, y='topic')
plt.title('토픽 길이 (공백 제거)')
plt.xlabel('Index')
plt.grid(True)

plt.tight_layout()
plt.show()

# 자음이나 모음만으로 구성된 경우 확인

In [ ]:
moeum_list = ['ㅏ', 'ㅓ', 'ㅗ', 'ㅜ', 'ㅡ', 'ㅣ', 'ㅐ', 'ㅔ', 'ㅚ', 'ㅟ', 
              'ㅑ', 'ㅕ', 'ㅛ', 'ㅠ', 'ㅒ', 'ㅖ', 'ㅘ', 'ㅙ', 'ㅝ', 'ㅞ', 'ㅢ']
jaeum_list = ['ㄱ', 'ㄴ', 'ㄷ', 'ㄹ', 'ㅁ', 'ㅂ', 'ㅅ', 'ㅇ', 'ㅈ', 'ㅊ', 'ㅋ', 
              'ㅌ', 'ㅍ', 'ㅎ', 'ㄲ', 'ㄸ', 'ㅃ', 'ㅆ', 'ㅉ']

print('\n# 대화에서 모음만으로 구성된 경우')
print('---'*25)
for moeum in moeum_list:
    temp = train_df[train_df['dialogue'].apply(lambda x: x.find(moeum) != -1)]['dialogue']
    if len(temp) > 0:
        print(f'[{moeum}]')
        print(temp)
print('---'*25)

print('\n# 대화에서 자음만으로 구성된 경우')
print('---'*25)
for jaeum in jaeum_list:
    temp = train_df[train_df['dialogue'].apply(lambda x: x.find(jaeum) != -1)]['dialogue']
    if len(temp) > 0:
        print(f'[{jaeum}]')
        print(temp)
print('---'*25)

print('\n# 요약문에서 모음만으로 구성된 경우')
print('---'*25)
for moeum in moeum_list:
    temp = train_df[train_df['summary'].apply(lambda x: x.find(moeum) != -1)]['summary']
    if len(temp) > 0:
        print(f'[{moeum}]')
        print(temp)
print('---'*25)

print('\n# 요약문에서 자음만으로 구성된 경우')
print('---'*25)
for jaeum in jaeum_list:
    temp = train_df[train_df['summary'].apply(lambda x: x.find(jaeum) != -1)]['summary']
    if len(temp) > 0:
        print(f'[{jaeum}]')
        print(temp)
print('---'*25)

In [ ]:
print(train_df.loc[9677, 'dialogue'])

# 마스킹 확인

## 개인정보

In [ ]:
def reg_masking(text):
    pattern = r'#\w+#'
    masked = re.findall(pattern, text)
    return masked

masked_train = train_df['dialogue'].apply(lambda x: str(set(reg_masking(x))))
for idx in range(0, train_df.shape[0]):
    print(masked_train[idx])

In [ ]:
temp_df = masked_train.reset_index()
temp_df['dialogue'] = temp_df['dialogue'].apply(lambda x: set(eval(x)))
unique_dialogues = set().union(*temp_df['dialogue'])
print(unique_dialogues)

## 사람

In [ ]:
def reg_person(text):
    pattern = r'#\w+\d#'
    masked = re.findall(pattern, text)
    return masked

masked_person = train_df['dialogue'].apply(lambda x: str(set(reg_person(x))))
for idx in range(0, train_df.shape[0]):
    print(masked_person[idx])

In [ ]:
temp_df = masked_person.reset_index()
temp_df['dialogue'] = temp_df['dialogue'].apply(lambda x: set(eval(x)))
unique_dialogues = set().union(*temp_df['dialogue'])
print(unique_dialogues)

# 데이터 전처리

In [ ]:
# 좌우 공백 제거
train_df['dialogue'] = train_df['dialogue'].apply(lambda x: x.strip())
train_df['summary'] = train_df['summary'].apply(lambda x: x.strip())
train_df['topic'] = train_df['topic'].apply(lambda x: x.strip())

# 자음, 모음만으로 구성된 경우 적절한 값으로 대체
train_df['dialogue'] = train_df['dialogue'].apply(lambda x: x.replace('제ㅏ', '제가'))
train_df['dialogue'] = train_df['dialogue'].apply(lambda x: x.replace('척했ㄷ거든', '척했거든'))
train_df['dialogue'] = train_df['dialogue'].apply(lambda x: x.replace('배경ㅇ로', '배경으로'))
train_df['dialogue'] = train_df['dialogue'].apply(lambda x: x.replace('ㅋㅋ', '웃기다'))
train_df['dialogue'] = train_df['dialogue'].apply(lambda x: x.replace('아직ㅍ알맞는', '아직 알맞는'))
train_df['summary'] = train_df['summary'].apply(lambda x: x.replace('머라이어 ㅐ리', '머라이어 캐리'))

# special_tokens는 학습할 때 수정

In [ ]:
train_df.to_csv('../../data/train_data-preprocessing.csv', index=False)